# Challenge Prompting

Resolver los siguientes ejercicios dejando el codigo con su ejecucion.

Importar las librerias necesarias y **correr las celdas para visualizar el resultado en cada ejercicio**.

In [10]:
## bloque importacion de librerias

import json
import ipywidgets as widgets
from IPython.display import display, clear_output

In [1]:
## bloque variables de entorno
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file

api_key = os.getenv("COHERE_API_KEY")
print(api_key)  # Verify the key is loaded


tEKdKO2dEbdOaL02QOJIb2bjhKaAAugqAE1uODtr


In [2]:
## bloque conexion a Cohere
import cohere
co = cohere.ClientV2()
# alternativa:
# co = cohere.ClientV2(api_key)

response = co.chat(
    model="command-r-plus-08-2024",
    messages=[{"role": "user", "content": "hello world!"}],
)

print(response.message.content[0].text)



Hello! How can I help you today?


## Ejercicio 1

Extraccion de entidades

Utilizar el LLM para extraer las siguientes entidades del texto medico.

- Paciente:
    - Nombre
    - Edad
- Fecha de admisión
- Síntomas
- Diagnóstico
- Tratamiento recomendado

**Aclaracion:** 

La salida tiene que ser un **string con formato de tipo json**, el cual se convertira en un diccionario de Python.

Si la linea de conversion en test da error el ejercicio no esta completo.

In [ ]:
# ejemplo 

# texto a analizar
"""La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""


# respuesta del LLM
{
  "paciente": {
    "nombre": "María González",
    "edad": 45
  },
  "fecha_admision": "2023-08-05",
  "sintomas": [
    "fatiga crónica",
    "dolores musculares"
  ],
  "diagnostico": "fibromialgia",
  "tratamiento": [
    "fisioterapia",
    "medicamentos analgésicos"
  ]
}

In [ ]:
#####

In [33]:
text_to_analize = """Sofía López, de 28 años, ingresó al Hospital Infantil el 3 de abril de 2023 debido a fiebre alta y tos persistente./
Después de varias pruebas, se le diagnosticó neumonía. La pediatra responsable, Dra. Claudia Torres, indicó tratamiento con antibióticos y reposo./
La próxima evaluación será el 10 de abril."""

In [34]:
# your code here

# 
system_prompt = """
se te proporcionará un texto medico, tu tarea es extraer las siguientes entidades: 
- paciente: nombre, edad(entero)
- fecha de admision (yyyy-MM-dd)
- sintomas
- Diagnostico
- Tratamiento 

en caso de ser varios sintomas o tratamientos devolver lista 
"""
prompt = f"""
    Extrae las entidades del siguiente texto medico: {text_to_analize}
"""

response = co.chat(
    model="command-r-plus-08-2024",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ],
    response_format= { "type": "json_object" }
)

llm_response = response.message.content[0].text


In [35]:
# test

final_result = json.loads(llm_response)

In [36]:
print(final_result)

{'paciente': {'nombre': 'Sofía López', 'edad': 28}, 'fecha_de_admision': '2023-04-03', 'sintomas': ['fiebre alta', 'tos persistente'], 'diagnostico': 'neumonía', 'tratamiento': ['antibióticos', 'reposo']}


## Ejercicio 2

Tenemos dos funciones en Python, una llamada *'add_contact'* y otra llamada *'get_information'*.

**Utilizar algun LLM que permita funtion calling** y desarrollar un codigo secuencial automatico que consiga:

Interpretar la consulta del usuario, identificar a que funcion llamar, luego llamarla (si es que aplica) y darle una respuesta final al usuario.  (usar function calling para esta solucion)

La entrada a dicho codigo es la consulta del usuario, a continuacion algunos ejemplos:

- "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."
- "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
- "Cual es el Email de Juan Pérez.?"

Salidas esperadas de dichos ejemplos (variaran porque las genera el LLM):
-  El contacto fue anadido con exito
-  Se anadio el contacto
-  El email de juan perez es juanperez@mail.com

Link de ayuda: https://github.com/cohere-ai/notebooks/blob/main/notebooks/agents/Vanilla_Tool_Use_v2.ipynb

In [30]:
contacts = {'Joaquin Lopez':{'phone': '15456663258', 'email': 'Joacolocolopez@gmail.com'},
                  'Flavio Oncativo':{'phone': '1545554178', 'email': 'FOncativo@hotmail.com'}
           }

In [18]:
def add_contact(name, phone, email):
    """
    Agrega un contacto al diccionario.
    Parámetros:
        name (str): Nombre del contacto.
        phone (str): Número de teléfono del contacto.
        email (str): Correo electrónico del contacto.
    Retorna:
        str: Mensaje confirmando la adición del contacto.
    """
    contacts[name] = {'phone': phone, 'email': email}
    return "Contacto añadido con éxito."

def get_information(name):
    """
    Recupera la información de un contacto.
    Parámetros:
        name (str): Nombre del contacto.
    Retorna:
        dict/str: Información del contacto o un mensaje si no existe.
    """
    if name in contacts:
        return contacts[name]
    else:
        return "Contacto no encontrado."

In [19]:
# mapa de funciones
functions_map = {
    "add_contact": add_contact,
    "get_information": get_information
}

In [ ]:
# Definicion de tools
tools = [
                       cohere.ToolV2(
            type="function",
            function={
        "name": "add_contact",
        "description": "Agrega un nuevo contacto al diccionario con su nombre, teléfono y correo electrónico.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Nombre del contacto (por ejemplo, 'Juan Pérez')."
                },
                "phone": {
                    "type": "string",
                    "description": "Número de teléfono del contacto en formato string (por ejemplo, '1234567890')."
                },
                "email": {
                    "type": "string",
                    "description": "Correo electrónico del contacto (por ejemplo, 'juanperez@gmail.com')."
                }
            },
            "required": ["name", "phone", "email"]
        }
    }),
    cohere.ToolV2(
            type="function",
            function={
        "name": "get_information",
        "description": "Obtiene la información de un contacto existente en el diccionario, incluyendo teléfono y correo.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Nombre del contacto que deseas buscar (por ejemplo, 'Flavio Oncativo')."
                }
            },
            "required": ["name"]
        }
    })
]

In [ ]:
# instrucciones al modelo
system_prompt = """
vas a ayudar a los usuarios a gestionar una lista de contactos Tu tarea es interpretar consultas en lenguaje natural relacionadas con la administración de contactos y ejecutar las acciones correspondientes.
Tienes dos funciones principales disponibles:
- add_contact: Para añadir un nuevo contacto con nombre, teléfono y correo electrónico
- get_information: Para obtener la información de un contacto existente
"""
# instruccion del usuario
prompt = "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
]

In [ ]:
co_tools = cohere.ClientV2()

response = co_tools.chat(
    model="command-r-plus-08-2024",
                messages=messages,
                   tools=tools)

# obtener el tool plan 
# verificacion de llamada al metodo correcto junto con sus parametros
print("The model recommends doing the following tool calls:\n")
print("Tool plan:")
print(response.message.tool_plan, "\n")
print("Tool calls:")
for tc in response.message.tool_calls:
    print(f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}")
# append the chat history
messages.append(
    {
        "role": "assistant",
        "tool_calls": response.message.tool_calls,
        "tool_plan": response.message.tool_plan,
    }
)

The model recommends doing the following tool calls:

Tool plan:
He guardado a Lucía Gómez en tus contactos. 

Tool calls:
Tool name: add_contact | Parameters: {"email":"lucia.gomez@gmail.com","name":"Lucía Gómez","phone":"555-5678"}


In [24]:
# Ejecutar funcione necesarias
for tc in response.message.tool_calls:
    tool_result = functions_map[tc.function.name](**json.loads(tc.function.arguments))
    tool_content = []
    for data in tool_result:
        tool_content.append({"type": "document", "document": {"data": json.dumps(data)}})
    # agregar al historial del chat
    messages.append(
        {"role": "tool", "tool_call_id": tc.id, "content": tool_content}
    )
    
    print("Tool results that will be fed back to the model in step 4:")
    for result in tool_content:
        print(result)


Tool results that will be fed back to the model in step 4:
{'type': 'document', 'document': {'data': '"C"'}}
{'type': 'document', 'document': {'data': '"o"'}}
{'type': 'document', 'document': {'data': '"n"'}}
{'type': 'document', 'document': {'data': '"t"'}}
{'type': 'document', 'document': {'data': '"a"'}}
{'type': 'document', 'document': {'data': '"c"'}}
{'type': 'document', 'document': {'data': '"t"'}}
{'type': 'document', 'document': {'data': '"o"'}}
{'type': 'document', 'document': {'data': '" "'}}
{'type': 'document', 'document': {'data': '"a"'}}
{'type': 'document', 'document': {'data': '"\\u00f1"'}}
{'type': 'document', 'document': {'data': '"a"'}}
{'type': 'document', 'document': {'data': '"d"'}}
{'type': 'document', 'document': {'data': '"i"'}}
{'type': 'document', 'document': {'data': '"d"'}}
{'type': 'document', 'document': {'data': '"o"'}}
{'type': 'document', 'document': {'data': '" "'}}
{'type': 'document', 'document': {'data': '"c"'}}
{'type': 'document', 'document': {'

In [ ]:
# TIPS
# Probar primero generando una funcion y llamarla, luego anadir la otra
# Plantearlo paso por paso en distintas celdas, analizar las salidas y las entradas, como identificamos a que funcion llamar?
# luego automatizar dentro de una sola celda


# Lo importante es entregar hasta donde lleguen, sea una funcion, las dos pero sin poder hacer el flujo automatico, lo que puedan, siempre y cuando este
# claro lo que se quizo hacer con comentarios.

In [ ]:
# Historial de mensajes donde se agrega el tool content
print(messages)

[{'role': 'system', 'content': '\nvas a ayudar a los usuarios a gestionar una lista de contactos Tu tarea es interpretar consultas en lenguaje natural relacionadas con la administración de contactos y ejecutar las acciones correspondientes.\nTienes dos funciones principales disponibles:\n- add_contact: Para añadir un nuevo contacto con nombre, teléfono y correo electrónico\n- get_information: Para obtener la información de un contacto existente\n'}, {'role': 'user', 'content': 'Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com'}, {'role': 'assistant', 'tool_calls': [ToolCallV2(id='add_contact_pnfgaprqe2pz', type='function', function=ToolCallV2Function(name='add_contact', arguments='{"email":"lucia.gomez@gmail.com","name":"Lucía Gómez","phone":"555-5678"}'))], 'tool_plan': 'He añadido a Lucía Gómez a tu lista de contactos.'}, {'role': 'tool', 'tool_call_id': 'add_contact_pnfgaprqe2pz', 'content': [{'type': 'document', 'document': {'data':

In [25]:
# Respuesta hacia el usuario
final_response = co_tools.chat(
    model="command-r-plus-08-2024",
    messages=messages,
    tools=tools
)
print("Final answer:")
print(final_response.message.content[0].text)

Final answer:
He guardado a Lucía Gómez en tus contactos.


In [ ]:
# Obtener informacion de contacto
nuevo_prompt = "Cual es el email de Joaquin Lopez"
messages.append(
        {"role": "user", "content": nuevo_prompt}
    )
new_response = co_tools.chat(
    model="command-r-plus-08-2024",
                messages=messages,
                   tools= tools)

print("The model recommends doing the following tool calls:\n")
print("Tool plan:")
print(new_response.message.tool_plan, "\n")
print("Tool calls:")
for tc in new_response.message.tool_calls:
    print(f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}")
# append the chat history
messages.append(
    {
        "role": "assistant",
        "tool_calls": new_response.message.tool_calls,
        "tool_plan": new_response.message.tool_plan,
    }
)
for tc in new_response.message.tool_calls:
    tool_result = functions_map[tc.function.name](**json.loads(tc.function.arguments))
    print(tool_result)
    tool_content = []
    for data in tool_result:
        tool_content.append({"type": "document", "document": {"data": json.dumps(data)}})
    # agregar al historial del chat
    messages.append(
        {"role": "tool", "tool_call_id": tc.id, "content": tool_content}
    )
    
    print("Tool results that will be fed back to the model in step 4:")
    for result in tool_content:
        print(result)



The model recommends doing the following tool calls:

Tool plan:
Buscaré el email de Joaquín López. 

Tool calls:
Tool name: get_information | Parameters: {"name":"Joaquín López"}
Contacto no encontrado.
Tool results that will be fed back to the model in step 4:
{'type': 'document', 'document': {'data': '"C"'}}
{'type': 'document', 'document': {'data': '"o"'}}
{'type': 'document', 'document': {'data': '"n"'}}
{'type': 'document', 'document': {'data': '"t"'}}
{'type': 'document', 'document': {'data': '"a"'}}
{'type': 'document', 'document': {'data': '"c"'}}
{'type': 'document', 'document': {'data': '"t"'}}
{'type': 'document', 'document': {'data': '"o"'}}
{'type': 'document', 'document': {'data': '" "'}}
{'type': 'document', 'document': {'data': '"n"'}}
{'type': 'document', 'document': {'data': '"o"'}}
{'type': 'document', 'document': {'data': '" "'}}
{'type': 'document', 'document': {'data': '"e"'}}
{'type': 'document', 'document': {'data': '"n"'}}
{'type': 'document', 'document': {'da

In [ ]:
# Pude hacer el function callin para la funcion de añadir contacto, pero no llegue a hacer la funcion get_information

## Ejercicio 3

Crear una funcion llamada "history_answer", que toma como parametro de entrada una pregunta sobre un contexto dado y la salida es la respuesta final del proceso impulsado por un LLM.

Dada una historia, el usuario podra hacer preguntas sobre la misma y el LLM debe responder siguiendo los siguientes lineamientos:

REQUISITOS DE LA RESPUESTA
- las respuestas deben ser en base a la historia
- ante la misma pregunta siempre debe responder de la misma manera.
- que responda en solo una oracion.
- el idioma que responde debe ser el mismo que con el que se pregunta (ingles, espanol, portugues).
- que agregue emojis en la oracion que resuman el contenido de la misma.
- que responda siempre en tercera persona.
- si la pregunta no tiene relacion alguna con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.
- Responder con 'Hakuna Matata!' al final de **todas** las respuestas (no importa idioma ni cantidad de tokens).

**Ayudin**: 
- No se limiten a usar 1 solo request al LLM, pueden dividirlo en partes para que por un lado se verifique el idioma, por otro lado se verifique si la pregunta tiene relacion con el contexto, etc

- Estructuren bien el prompt procurando separar instrucciones, contexto(historia) y pregunta del usuario.

- Recuerden usar el system message y user message.

In [ ]:
# ejemplo flojo de estructura de prompt
# prompt = f"Responde a la pregunta: {pregunta} de manera concisa y divertida en base a la siguiente historia: {historia}"

In [48]:
historia = """En un pequeño feudo medieval, Thomas, un joven campesino de dieciséis años, trabajaba desde el amanecer en los campos de trigo del señor feudal. El sol apenas había salido cuando él ya había arado más de lo que sus manos podían soportar. La vida era dura, pero su familia dependía de la cosecha para pagar los impuestos y mantener su hogar de madera y paja.

Un día, el feudo fue sacudido por noticias de guerra. El rey había llamado a todos los hombres en edad de luchar. Thomas sabía que, al igual que otros jóvenes, no tenía elección. Cambió la hoz por una lanza rudimentaria y se unió a la milicia local. Sin entrenamiento, fue empujado a un campo de batalla embarrado, donde el acero resonaba y los gritos de los hombres llenaban el aire.

La batalla fue un caos. Thomas, con el corazón latiendo en su pecho como un tambor de guerra, apenas podía distinguir amigo de enemigo. Logró esquivar una espada, pero cayó al suelo, cubierto de lodo y sangre. Levantándose, vio cómo un compañero caía junto a él, sus ojos abiertos, vacíos.

Cuando la batalla terminó, el silencio era tan profundo como el vacío que sentía. Thomas regresó al feudo, diferente, marcado por la muerte y la violencia. Su madre lo recibió con lágrimas en los ojos, pero él, con la mirada fija en el horizonte, sabía que la inocencia había quedado atrás, enterrada en aquel campo de batalla. La paz del feudo ya no era la misma; él tampoco."""



In [83]:
def history_answer(pregunta):
    # your code here
    system_prompt = "Tu tarea es responder las preguntas de acuerdo al contexto proporcionado"

    prompt = f""" 
    ### Instrucciones
    - Responder en base a la historia
    - Responde en una sola oracion
    - Responde en el idioma en el que es la pregunta. (si la pregunta es en español, tu respuesta sera en español. Si la pregunta es en ingles respondes en ingles)
    - Si la pregunta no tiene relacion con el contexto responde "Lo siento no puedo ayudarte con eso."
    - Al final de la respuesta agruega "Hakuna Matata!"
    - Agrega emojis en la oracion que resuman el contenido de la misma.
    - Respondr siempre en tercera persona.
    ### 
    contexto: {historia}
    ###
    pregunta: {pregunta}
    """

    response = co.chat(
    model="command-r-plus-08-2024",
    messages=[
        {"role": "system", "content":system_prompt},
        {"role": "system", "content":"Responde en el idioma en el que pregunta el usuario,  si la pregunta es en español respondes en español, si la pregunta es en ingles respondes en ingles, si la pregunta es en portugues respondes en portugues"},
        {"role": "user", "content": prompt}
    ],
        seed = 10
        
    )
    respuesta_al_usuario = response.message.content[0].text

    return respuesta_al_usuario

In [ ]:
# primera iteracion con algunas instrucciones
pregunta = "Como era la vida de Thomas antes de la guerra?"
# respuesta
print(history_answer(pregunta))

Thomas, un joven campesino, trabajaba arduamente en los campos de trigo desde el amanecer, soportando una vida dura para mantener a su familia.


In [ ]:
# pregunta fuera de contexto sin instruccion de respuesta
pregunta = "Quien es San Martin?"

# respuesta
print(history_answer(pregunta))

San Martín no es mencionado en esta historia.


In [59]:
# Respuesta a pregunta fuera de contexto y Hakuna Matata!
pregunta = "Quien es San Martin?"

# respuesta
print(history_answer(pregunta))

Lo siento, no puedo ayudarte con eso. Hakuna Matata!


In [75]:
# Respuesta en respectivo idioma
pregunta = "quem é Tomás?"

# respuesta
print(history_answer(pregunta))

Thomas é um jovem camponês que, aos dezesseis anos, troca a foice pelo campo de batalha, enfrentando a guerra e perdendo a inocência. 🌾🗡️👦 Hakuna Matata!


In [82]:
# seed diferente
pregunta = "quem é Tomás?"

# respuesta
print(history_answer(pregunta))

️ Thomas é um jovem camponês que, aos 16 anos, trocou o arado pela lança, lutando em uma guerra que mudou sua vida para sempre. 🌾🗡️ Hakuna Matata!


In [84]:
# Misma seed (10)
pregunta = "quem é Tomás?"

# respuesta
print(history_answer(pregunta))

Thomas é um jovem camponês que, aos dezesseis anos, troca a foice pelo campo de batalha, enfrentando a guerra e perdendo a inocência. 🌾⚔️💔 Hakuna Matata!


In [85]:
# Seed 10
pregunta = "quem é Tomás?"

# respuesta
print(history_answer(pregunta))

Thomas é um jovem camponês que, aos dezesseis anos, troca a foice pelo campo de batalha, enfrentando a guerra e perdendo a inocência. 🌾⚔️💔 Hakuna Matata!


## Ejercicio 4

Crear un chatbot sencillo impulsado por un LLM. 

Dicho bot esta destinado a un usuario final y debe cumplir las siguientes **condiciones en sus respuestas**:

- Responder en no mas de 70 tokens.
- Responder de manera positiva, con un tono entusiasta.
- Responder con consejos útiles, como si fueras un tutor.

 
**Otras consideraciones**:

Respetar el formato de la interfaz provista por el ejercicio.

Ademas agregar al codigo propuesto un historial de conversaciones para que el bot pueda mantener el hilo de lo que se esta hablando. Para probar no usen mas de 3 conversaciones anidadas para no enviarle tantos tokens.

Dejar impreso en el notebook el historial de la conversacion.

In [96]:
cb = cohere.ClientV2(api_key)

In [ ]:
# Crear widgets de entrada y salida
input_box = widgets.Text(placeholder='Escribe tu mensaje aquí...')
send_button = widgets.Button(description='Enviar')
output_box = widgets.Output()


# instrucciones de alto nivel 
system_prompt_tutor = """
### Instrucciones: 
-Responde de manera positiva y entusiasta
- Toma el rol de tutor, responderas con consejos utiles
- respuestas concisas y cortas
"""
# historial del chat para que el modelo pueda responder en contexto
chat_history = [
    {"role": "system", "content":system_prompt_tutor},
    {"role": "user", "content": "¿Cuál es el área de un triángulo?"}, 
    {"role": "assistant", "content": "El área de un triángulo se calcula con la fórmula: Área = (base * altura) / 2."},
    {"role": "user", "content": "¿Qué es un círculo?"}, 
    {"role": "assistant", "content": "Un círculo es una figura geométrica en la que todos los puntos están a la misma distancia de un punto central, llamado centro."}
            
]
# Función de respuesta simulada del chatbot
def chatbot_response(message):
    # Aquí puedes conectar tu modelo o lógica de chatbot real
    query = {"role": "user", "content": message}
    chat_history.append(query)
    response = cb.chat(
        model="command-r-plus-08-2024",
        messages=chat_history,
        # seteo de tokens maximos
        max_tokens = 70
    )
    chat_history.append(response.message)
    return response.message.content[0].text
    

# Función de manejo del botón
def on_send_button_clicked(b):
    with output_box:
        clear_output(wait=True)
        user_message = input_box.value
        if user_message.strip():
            print(f"Tú: {user_message}")
            response = chatbot_response(user_message)
            print(f"Chatbot: {response}")
        input_box.value = ''

# Asociar función al botón
send_button.on_click(on_send_button_clicked)

# Mostrar widgets
display(input_box, send_button, output_box)

Text(value='', placeholder='Escribe tu mensaje aquí...')

Button(description='Enviar', style=ButtonStyle())

Output()

In [ ]:

# Mostrar historial del chat completo
print(chat_history)

[{'role': 'system', 'content': '\n### Instrucciones: \n-Responde de manera positiva y entusiasta\n- Toma el rol de tutor, responderas con consejos utiles\n- respuestas concisas y cortas\n'}, {'role': 'user', 'content': '¿Cuál es el área de un triángulo?'}, {'role': 'assistant', 'content': 'El área de un triángulo se calcula con la fórmula: Área = (base * altura) / 2.'}, {'role': 'user', 'content': '¿Qué es un círculo?'}, {'role': 'assistant', 'content': 'Un círculo es una figura geométrica en la que todos los puntos están a la misma distancia de un punto central, llamado centro.'}, {'role': 'user', 'content': 'y como calculo el area de un circulo?'}, AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='El área de un círculo se calcula con la fórmula: Área = π * radio^2. Donde π (pi) es una constante aproximadamente igual a 3.14159.')], citations=None)]


### RECOMENDACIONES GENERALES

No se confien probando con un par de respuestas y ya, hagan minimo 5 pruebas por ejercicio para asi tener mas chances de visualizar errores en la generacion del contenido.

Prueben combinar LLMs con programacion convencional para los casos que vean convenientes (decisiones if else, respuestas estaticas, etc)

Prueben con distintos modelos de Cohere, hay algunos optimizados para ciertas aplicaciones.